# Dataset

### 이미지 불러오기

In [1]:
import os
import cv2


# 폴더 경로에 존재하는 이미지 파일을 읽어오는 함수
def load_images_from_folder(dir_path):
    images = []
    labels = []
    label = dir_path.split('/')[-1] # 'cat' 또는 'dog'
    for file_name in os.listdir(dir_path):
        image_path = os.path.join(dir_path, file_name) # 이미지 파일 경로
        image = cv2.imread(image_path) # 이미지 읽어오기
        image = cv2.resize(image, (32, 32)) # 이미지의 크기를 (32, 32)로 조정
        image = image.reshape(-1) # 이미지를 1차원으로 변경
        images.append(image)
        labels.append(label)

    return images, labels

In [2]:
cat_images, cat_labels = load_images_from_folder('../data/cifar10_images/cat')
dog_images, dog_labels = load_images_from_folder('../data/cifar10_images/dog')

print(f"이미지 모양: {cat_images[0].shape}")
print(f"고양이 데이터 갯수: {len(cat_labels)}")
print(f"강아지 데이터 갯수: {len(dog_labels)}")

이미지 모양: (3072,)
고양이 데이터 갯수: 100
강아지 데이터 갯수: 100


### 레이블 벡터화

In [3]:
import numpy as np

# 데이터와 레이블 병함
images = np.array(cat_images + dog_images)
labels = np.array(cat_labels + dog_labels)

# 레이블 -> 벡터
label_to_index = {'cat': 0, 'dog': 1}
indexed_labels = np.array([label_to_index[label] for label in labels])

print(f"벡터화된 레이블: {indexed_labels[:10]}")

벡터화된 레이블: [0 0 0 0 0 0 0 0 0 0]


### 학습 데이터와 테스트 데이터 나누기

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, indexed_labels, test_size=0.2, random_state=42)

print(f"학습 데이터 갯수: {len(y_train)}")
print(f"테스트 데이터 갯수: {len(y_test)}")

학습 데이터 갯수: 160
테스트 데이터 갯수: 40


# Distance Matrix

In [5]:
# L1 거리 계산 함수
def l1_distance(a, b):
    return np.sum(np.abs(a - b))

In [6]:
# L2 거리 계산 함수
def l2_distance(a, b):
    return np.sqrt(np.sum((a - b)**2))

# K-Nearest Neighbor

In [18]:
from collections import Counter


class KNearestNeighbor:
    def __init__(self, k=3, metric='l2'):
        self.k = k
        if metric == 'l1':
            self.distance_matrix = l1_distance
        elif metric == 'l2':
            self.distance_matrix = l2_distance
        else:
            raise ValueError("지원하지 않는 함수입니다. l1과 l2 중에서 선택해주세요.")
    
    def train(self, x, y):
        # 입력 받은 학습 데이터를 저장
        self.x_train_list = x
        self.y_train_list = y
    
    def predict(self, inputs):
        # 입력 받은 데이터와 가지고 있는 학습 데이터를 비교하여 상위 k개의 비율을 측정하여 결과 도출
        predictions = []
        for input in inputs:
            distances = [self.distance_matrix(input, x_train) for x_train in self.x_train_list]
            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = [self.y_train_list[i] for i in k_indices]
            most_common = Counter(k_nearest_labels).most_common(1)
            predictions.append(most_common[0][0])
        
        return predictions

### Train

In [25]:
knn = KNearestNeighbor(k=5, metric='l2') # k를 1로 설정하면 Nearest_Neighbor가 된다.
knn.train(x_train, y_train)

### Predict

In [26]:
from sklearn.metrics import accuracy_score

knn_predictions = knn.predict(x_test)
print(f"실제 값의 일부: {y_test[:5]}")
print(f"예측 값의 일부: {knn_predictions[:5]}")
print(f"Nearest Neighbor의 정확도: {accuracy_score(y_test, knn_predictions)}")

실제 값의 일부: [0 0 0 1 1]
예측 값의 일부: [1, 0, 1, 0, 1]
Nearest Neighbor의 정확도: 0.525


# scikit learn

In [31]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(x_train, y_train) # 학습
nn_predictions = knn.predict(x_test)

knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(x_train, y_train) # 학습
knn_predictions = knn.predict(x_test)

# 정확도 계산
nn_accuracy = accuracy_score(y_test, nn_predictions)
knn_accuracy = accuracy_score(y_test, knn_predictions)

print("Nearest Neighbor의 정확도 (k=1):", nn_accuracy)
print("K-Nearest Neighbor의 정확도 (k=5):", knn_accuracy)

Nearest Neighbor의 정확도 (k=1): 0.55
K-Nearest Neighbor의 정확도 (k=5): 0.5
